MODEL PREDICTION

In [4]:
from __future__ import annotations

import os
from datetime import date, timedelta

import numpy as np
import pandas as pd
from prophet import Prophet

# ========= CONFIG =========
dataset = pd.read_excel("2025 Allianz Datathon Dataset.xlsx", sheet_name="Visitation Data")
dataset.to_csv("visit_data.csv", index=False)
INPUT_CSV  = "visit_data.csv"   # columns: Year, Week, <Resort1> ... <ResortN>
SEASON_WEEKS = 15               # fixed
OUT_DIR = "outputs"
os.makedirs(OUT_DIR, exist_ok=True)

# Shape model calendar inputs (tune as needed)
USE_COUNTRY_HOLIDAYS = True
COUNTRY_NAME = "Australia"      # try "Australia"; if unavailable in your Prophet version, it will fall back to "AU" or skip.
CUSTOM_HOLIDAYS_DF = None       # optionally pass a DataFrame with columns ["holiday","ds","lower_window","upper_window"]
# REGRESSOR_COLS: list[str] = []  # e.g., ["school_break_fraction"]
REGRESSOR_MODE = "multiplicative"
# ==========================


# ---------- Week → date mapping (calendar‑aware) ----------
# Your definition: Week 1 is exactly 9‑June of that year, Week k = Week1 + 7*(k-1) days.
def week_anchor_date(year: int, week: int) -> pd.Timestamp:
    assert 1 <= week <= SEASON_WEEKS
    start = date(year, 6, 9)  # exactly 9‑Jun (no “make Monday” shift)
    return pd.Timestamp(start + timedelta(days=7*(week - 1)))

def season_calendar(year: int, weeks: int = SEASON_WEEKS) -> pd.DatetimeIndex:
    start = date(year, 6, 9)
    return pd.to_datetime([start + timedelta(days=7*i) for i in range(weeks)])


# ---------- Load & reshape ----------
raw = pd.read_csv(INPUT_CSV)
raw.columns = [c.strip() for c in raw.columns]
assert {"Year","Week"}.issubset(set(raw.columns)), "CSV must have Year and Week."

# resort columns are everything except Year/Week and stray unnamed cols
resort_cols = [c for c in raw.columns if c not in {"Year","Week"} and not c.startswith("Unnamed")]
if not resort_cols:
    raise ValueError("No resort columns detected.")

# build ds from Year/Week using your fixed weekly anchors
raw["ds"] = pd.to_datetime([week_anchor_date(int(y), int(w)) for y, w in zip(raw["Year"], raw["Week"])])
# Long format: one row per resort-week
long = raw.melt(id_vars=["Year","Week","ds"], value_vars=resort_cols,
                var_name="resort", value_name="y").sort_values(["resort","ds"])

# ---------- Proportions (shape targets) ----------
season_totals_obs = (
                    long.groupby(["resort","Year"], as_index=False)["y"]
                        .sum()
                        .rename(columns={"y":"season_total_obs"})
                    )
long = long.merge(season_totals_obs, on=["resort","Year"], how="left")
long["prop"] = long["y"] / long["season_total_obs"].replace(0, np.nan)

# ---------- Shape model on REAL dates ----------
def _add_holidays_safe(m: Prophet) -> None:
    if not USE_COUNTRY_HOLIDAYS:
        return
    # Try a couple of country identifiers; ignore if unsupported
    for cn in [COUNTRY_NAME, "AU", "Australia"]:
        if not cn:
            continue
        try:
            m.add_country_holidays(country_name=cn)
            return
        except Exception:
            continue
    # If none succeeded, we silently proceed without PHs.


def fit_shape_model_real_dates(df_resort: pd.DataFrame,
                               regressor_mode: str = "multiplicative") -> Prophet:
    """
    Train Prophet on weekly proportions using REAL calendar dates (ds).
    No trend: we want a stable within‑season shape, modulated by holidays/regressors.
    """
    train = df_resort[["ds","prop"]].copy()
    train = train.rename(columns={"prop":"y"})

    m = Prophet(
        yearly_seasonality=10,       # allows a date‑anchored within‑year pattern
        weekly_seasonality=False,      # already at weekly granularity
        daily_seasonality=False,
        seasonality_mode="multiplicative",
        n_changepoints=0,              # forbid trend in shape
        changepoint_prior_scale=0.001,
        seasonality_prior_scale=5.0
    )

    #add national holidays
    _add_holidays_safe(m)
    
    m.fit(train)
    return m


def forecast_shape_for_year_real_dates(m_shape: Prophet,
                                       year: int,
                                       weeks: int,
                                       ) -> pd.DataFrame:
    """
    Forecast weekly proportions for a season on REAL calendar dates.
    If you have regressor_cols, you must supply those columns in future_regressors
    (aligned on 'ds'). Missing columns are filled with zeros.
    """
    fut = pd.DataFrame({"ds": season_calendar(year, weeks)})
    # Merge any provided future regressors (must include 'ds'); otherwise fill zeros.

    fc = (
        m_shape.predict(fut)[["ds","yhat","yhat_lower","yhat_upper"]]
        .rename(columns={"yhat":"prop_hat","yhat_lower":"prop_lo","yhat_upper":"prop_hi"})
    )

    # Valid proportions: non‑negative and sum to 1
    for c in ["prop_hat","prop_lo","prop_hi"]:
        fc[c] = fc[c].clip(lower=0)
        s = fc[c].sum()
        fc[c] = fc[c] / s if s > 0 else np.nan

    fc["Week"] = np.arange(1, weeks+1)
    return fc


# ---------- Annual totals (scale) ----------
annual = long.groupby(["resort","Year"], as_index=False)["y"].sum().rename(columns={"y":"T"})

def fit_total_model(df_resort_annual: pd.DataFrame) -> Prophet:
    train = df_resort_annual.rename(columns={"Year":"ds","T":"y"}).copy()
    train["ds"] = pd.to_datetime(train["ds"].astype(str) + "-01-01")
    m = Prophet(
        yearly_seasonality=False,     # not meaningful for 1/yr sampling
        weekly_seasonality=False,
        daily_seasonality=False,
        seasonality_mode="multiplicative",
        n_changepoints=2,
        changepoint_prior_scale=0.1,
    )
    m.fit(train)
    return m

def forecast_total_for_year(m_total: Prophet, year:int) -> pd.DataFrame:
    fut = pd.DataFrame({"ds": [pd.Timestamp(f"{year}-01-01")]})
    fc = m_total.predict(fut)[["ds","yhat","yhat_lower","yhat_upper"]].rename(
        columns={"yhat":"T_hat","yhat_lower":"T_lo","yhat_upper":"T_hi"}
    )
    fc["Year"] = year
    return fc


# ---------- Train per resort, recompose ----------
forecast_2026_all = []

for resort, dfr in long.groupby("resort"):
    # --- SHAPE on real dates with optional holidays/regressors
    m_shape = fit_shape_model_real_dates(
        dfr,
        regressor_mode=REGRESSOR_MODE
    )

    future_regs_2026 = None
    shape_2026 = forecast_shape_for_year_real_dates(
        m_shape, year=2026, weeks=SEASON_WEEKS
    )

    # --- TOTALS model on annual sums
    dfr_annual = annual[annual["resort"] == resort].copy()
    m_total = fit_total_model(dfr_annual)

    # 2026 absolute weekly = shape × total
    fc_total_2026 = forecast_total_for_year(m_total, 2026)
    T_hat_2026, T_lo_2026, T_hi_2026 = map(float, fc_total_2026[["T_hat","T_lo","T_hi"]].iloc[0])

    fc2026 = shape_2026.copy()
    fc2026["resort"] = resort
    
    # Recompose with uncertainty bounds (simple product ignoring covariance)
    fc2026["y_abs_hat"] = fc2026["prop_hat"] * T_hat_2026
    fc2026["y_abs_lo"]  = fc2026["prop_lo"]  * T_lo_2026
    fc2026["y_abs_hi"]  = fc2026["prop_hi"]  * T_hi_2026
    fc2026["Year"] = 2026
    forecast_2026_all.append(fc2026[["resort","ds","Year","Week","prop_hat","prop_lo","prop_hi","y_abs_hat","y_abs_lo","y_abs_hi"]])

# ---------- Concatenate & Save ----------
forecast_2026 = pd.concat(forecast_2026_all, ignore_index=True).sort_values(["resort","ds"])

p2026 = os.path.join(OUT_DIR, "two_stage_forecast_2026_by_resort.csv")

# filled_2025.to_csv(p2025, index=False)
forecast_2026.to_csv(p2026, index=False)

print("Saved:", p2026)

13:06:08 - cmdstanpy - INFO - Chain [1] start processing
13:06:08 - cmdstanpy - INFO - Chain [1] done processing
13:06:08 - cmdstanpy - ERROR - Chain [1] error: error during processing Operation not permitted
Optimization terminated abnormally. Falling back to Newton.
13:06:08 - cmdstanpy - INFO - Chain [1] start processing
13:06:09 - cmdstanpy - INFO - Chain [1] done processing
13:06:09 - cmdstanpy - INFO - Chain [1] start processing
13:06:09 - cmdstanpy - INFO - Chain [1] done processing
13:06:09 - cmdstanpy - INFO - Chain [1] start processing
13:06:09 - cmdstanpy - INFO - Chain [1] done processing
13:06:09 - cmdstanpy - ERROR - Chain [1] error: error during processing Operation not permitted
Optimization terminated abnormally. Falling back to Newton.
13:06:09 - cmdstanpy - INFO - Chain [1] start processing
13:06:09 - cmdstanpy - INFO - Chain [1] done processing
13:06:09 - cmdstanpy - INFO - Chain [1] start processing
13:06:09 - cmdstanpy - INFO - Chain [1] done processing
13:06:09 -

Saved: outputs\two_stage_forecast_2026_by_resort.csv


MODEL EVALUATION

In [5]:
import numpy as np, pandas as pd

def jensen_shannon(p, q, eps=1e-12):
    p = np.clip(p, eps, 1); q = np.clip(q, eps, 1)
    p /= p.sum(); q /= q.sum(); m = 0.5*(p+q)
    return 0.5*(np.sum(p*np.log(p/m)) + np.sum(q*np.log(q/m))) ** 0.5

def evaluate_shape_loyo(long: pd.DataFrame):
    rows = []
    for resort, dfR in long.groupby("resort"):
        for yr, dfY in dfR.groupby("Year"):
            # actual proportions by week (sum to 1 within year)
            tgt = dfY.sort_values("ds")["y"].to_numpy().astype(float)
            tgt = tgt / tgt.sum() if tgt.sum()>0 else np.full_like(tgt, np.nan)

            # train on other years
            train = dfR[dfR["Year"] < yr].copy()
            if train["Year"].nunique() < 2:  # need at least 2 seasons to generalize
                continue
            m = fit_shape_model_real_dates(
                pd.DataFrame({"ds":train["ds"], "prop":(train["y"]/train.groupby("Year")["y"].transform("sum")).values})
            )
            fut = pd.DataFrame({"ds": dfY.sort_values("ds")["ds"].values})
            pred = m.predict(fut)["yhat"].clip(lower=0).to_numpy()
            pred = pred / pred.sum() if pred.sum()>0 else np.full_like(pred, np.nan)

            mae = np.nanmean(np.abs(pred - tgt))
            rmse = np.sqrt(np.nanmean((pred - tgt)**2))
            jsd = jensen_shannon(pred, tgt)
            rows.append({"resort":resort,"year":yr,"shape_mae":mae,"shape_rmse":rmse,"shape_jsd":jsd})
    return pd.DataFrame(rows)


In [6]:
# df = pd.read_csv("./outputs/two_stage_forecast_2026_by_resort.csv")
# df.head()
eval_long = long[long["Year"] != 2025] #2025 not complete data
evaluation = evaluate_shape_loyo(long)
evaluation


13:06:38 - cmdstanpy - INFO - Chain [1] start processing
13:06:39 - cmdstanpy - INFO - Chain [1] done processing
13:06:39 - cmdstanpy - INFO - Chain [1] start processing
13:06:50 - cmdstanpy - INFO - Chain [1] done processing
13:06:50 - cmdstanpy - INFO - Chain [1] start processing
13:06:56 - cmdstanpy - INFO - Chain [1] done processing
13:06:57 - cmdstanpy - INFO - Chain [1] start processing
13:06:57 - cmdstanpy - INFO - Chain [1] done processing
13:06:57 - cmdstanpy - INFO - Chain [1] start processing
13:06:57 - cmdstanpy - INFO - Chain [1] done processing
13:06:57 - cmdstanpy - INFO - Chain [1] start processing
13:06:57 - cmdstanpy - INFO - Chain [1] done processing
13:06:57 - cmdstanpy - ERROR - Chain [1] error: error during processing Operation not permitted
Optimization terminated abnormally. Falling back to Newton.
13:06:57 - cmdstanpy - INFO - Chain [1] start processing
13:06:57 - cmdstanpy - INFO - Chain [1] done processing
13:06:57 - cmdstanpy - INFO - Chain [1] start process

,resort,year,shape_mae,shape_rmse,shape_jsd
0,Charlotte Pass,2016,0.011437,0.014952,0.065714
1,Charlotte Pass,2017,0.006762,0.008008,0.036009
2,Charlotte Pass,2018,0.006458,0.008169,0.035577
3,Charlotte Pass,2019,0.007072,0.008823,0.038382
4,Charlotte Pass,2020,0.053209,0.063426,0.284519
...,...,...,...,...,...
85,Thredbo,2021,0.037788,0.046670,0.171115
86,Thredbo,2022,0.012640,0.015636,0.066573
87,Thredbo,2023,0.008034,0.010592,0.046663
88,Thredbo,2024,0.016389,0.018435,0.083369


In [7]:
# Show all rows
pd.set_option("display.max_rows", None)

# Show all columns
pd.set_option("display.max_columns", None)

# Prevent columns from getting cut off horizontally
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", None)

evaluation.head(100)

,resort,year,shape_mae,shape_rmse,shape_jsd
0,Charlotte Pass,2016,0.011437,0.014952,0.065714
1,Charlotte Pass,2017,0.006762,0.008008,0.036009
2,Charlotte Pass,2018,0.006458,0.008169,0.035577
3,Charlotte Pass,2019,0.007072,0.008823,0.038382
4,Charlotte Pass,2020,0.053209,0.063426,0.284519
5,Charlotte Pass,2021,0.037603,0.046277,0.169448
6,Charlotte Pass,2022,0.012844,0.015937,0.067725
7,Charlotte Pass,2023,0.008170,0.010917,0.048480
8,Charlotte Pass,2024,0.016275,0.018506,0.083361
9,Charlotte Pass,2025,0.013148,0.018352,0.051290


Visitor Amount Model Evaluation

In [24]:
import pandas as pd
import numpy as np
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Load data
df = pd.read_csv("visit_data.csv")
# Aggregate weekly into annual totals per resort
resort_cols = [c for c in df.columns if c not in {"Year","Week"}]
annual = df.groupby("Year")[resort_cols].sum().reset_index()
print()
# Storage for results
rows = []

for resort in resort_cols:
    series = annual[["Year", resort]].dropna()
    series = series.set_index("Year")[resort].sort_index()

    # For each possible forecast year
    for target_year in series.index:
        if target_year >= 2025:  # skip 2025 (future)
            continue

        # Train set = all years before target_year
        train = series.loc[series.index < target_year]

        # Only proceed if at least 5 years of history
        if len(train) < 9:
            continue

        try:
            # Fit ETS (additive trend, damped)
            model = ExponentialSmoothing(
                train,
                trend="add",
                seasonal=None,
                damped_trend=True
            ).fit(optimized=True)

            pred = model.forecast(1).iloc[0]
            actual = series.loc[target_year]

            mae = abs(pred - actual)
            rmse = np.sqrt((pred - actual)**2)
            mape = abs((pred - actual) / actual) * 100 if actual != 0 else np.nan

            rows.append({
                "resort": resort,
                "year": target_year,
                "actual": actual,
                "pred": pred,
                "mae": mae,
                "rmse": rmse,
                "mape": mape
            })
        except Exception as e:
            print(f"Skipping {resort} {target_year} due to error: {e}")

# Collect results
results = pd.DataFrame(rows)

# Average metrics per resort
metrics = results.groupby("resort")[["mae","rmse","mape"]].mean().reset_index()

print("Per-forecast results:\n", results.head())
print("\nAverage metrics per resort:\n", metrics)


c:\Users\richa\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
c:\Users\richa\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
c:\Users\richa\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:837: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
c:\Users\richa\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarn

Per-forecast results:
          resort  year  actual           pred            mae           rmse  \
0   Mt. Baw Baw  2023   96854   67266.912358   29587.087642   29587.087642   
1   Mt. Baw Baw  2024   76964   77569.327144     605.327144     605.327144   
2  Mt. Stirling  2023   10959   13094.625343    2135.625343    2135.625343   
3  Mt. Stirling  2024    9846    8522.872299    1323.127701    1323.127701   
4    Mt. Hotham  2023  398746  281370.888912  117375.111088  117375.111088   

        mape  
0  30.548132  
1   0.786507  
2  19.487411  
3  13.438226  
4  29.436060  

Average metrics per resort:
            resort           mae          rmse       mape
0  Charlotte Pass   2961.551769   2961.551769   8.735773
1     Falls Creek  92498.114652  92498.114652  21.126337
2     Mt. Baw Baw  15096.207393  15096.207393  15.667319
3      Mt. Buller  68667.760898  68667.760898  14.564580
4      Mt. Hotham  91385.547440  91385.547440  27.532882
5    Mt. Stirling   1729.376522   1729.376522 

c:\Users\richa\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:837: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
c:\Users\richa\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
c:\Users\richa\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
c:\Users\richa\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:837: FutureWar

In [25]:
results

,resort,year,actual,pred,mae,rmse,mape
0,Mt. Baw Baw,2023,96854,67266.912358,29587.087642,29587.087642,30.548132
1,Mt. Baw Baw,2024,76964,77569.327144,605.327144,605.327144,0.786507
2,Mt. Stirling,2023,10959,13094.625343,2135.625343,2135.625343,19.487411
3,Mt. Stirling,2024,9846,8522.872299,1323.127701,1323.127701,13.438226
4,Mt. Hotham,2023,398746,281370.888912,117375.111088,117375.111088,29.436060
5,Mt. Hotham,2024,255157,320552.983792,65395.983792,65395.983792,25.629704
6,Falls Creek,2023,439826,339438.018042,100387.981958,100387.981958,22.824476
7,Falls Creek,2024,435492,350883.752654,84608.247346,84608.247346,19.428198
8,Mt. Buller,2023,481230,377952.132841,103277.867159,103277.867159,21.461228
9,Mt. Buller,2024,444157,410099.345363,34057.654637,34057.654637,7.667932
